In [22]:
""" Template for GSA prediction exercise """

import numpy as np
import pandas as pd
from pmdarima import auto_arima
from statsmodels.tsa.statespace.varmax import VARMAX
import warnings
import time
import pickle

from calculate_returns import get_dates, cal_returns
from tests import test_adf, cointegration_test

def modelEstimate(trainingFilename):
    """
    Fit a model using historical data.

    Args:
        trainingFilename (str): path to training data. The data will
            be in the same format as the supplied `data.csv` file

    Returns:
        parameters (any): fitted model and any additional parameters
            you need to pass to modelForecast

    """
    train = pd.read_csv(trainingFilename)
    # calculate the returns of X using the underlying method of calculating 'returns'
    get_dates(train, 'timestamp')
    yreturns = cal_returns(train, 'date', 'yprice')
    if sum(train['returns'] != yreturns)>0:
        print(sum(train['returns'] != yreturns))
        raise ValueError('The given data set uses a different method to calculate returns!')
    else:
        print('The method to calculate xreturns seems to be the same as the underlying method to calculate returns.')
    train['xreturns'] = cal_returns(train, 'date', 'xprice')
    print('before testing')
    # test stationarity of returns and xreturns
    # p-value threshold = 0.05
    if test_adf(train['xreturns']) > 0.05:
        raise ValueError('The calculated returns of X is not stationary (ADF test)!')
    else:
        print('The calculated xreturns passes the ADF test.')
    if test_adf(train['returns']) > 0.05:
        raise ValueError('The given returns of Y is not stationary (ADF test)!')
    else:
        print('The given returns passes the ADF test.')
    print('after testing')
    df = train[['returns', 'xreturns']] # the given train set, to be splitted into train_df and val_df
    train_size = int(df.shape[0] * 0.8)
    train_df = df[0:train_size] # train set 
    val_df = df[train_size:].reset_index(drop=True) # validation set
    print('df defined')
    # check results of Johansen cointegration test
    if cointegration_test(df) == False:
        raise ValueError('The dataframe ["returns", "xreturns"] does not pass the Johansen cointegration test!')
    else:
        print('The df ["returns", "xreturns"] passes the Johansen cointegration test.')
    
    # get possible p, q values from auto_arima stepwise model
    # note that I set d=0, assuming stationary data (verified above by ADF test)
    pq = [] 
    for name, column in train_df.iteritems():
        print(f'In auto_arima, searching order of p and q for : {name}')
        stepwise_model = auto_arima(train_df[name], d=0, start_p=1, start_q=1, max_p=4, max_q=4, seasonal=False,
            trace=True,error_action='ignore',suppress_warnings=True, stepwise=True,maxiter=50)
        parameter = stepwise_model.get_params().get('order')
        print(f'optimal order for:{name} is: {parameter} \n\n')
        pq.append(stepwise_model.get_params().get('order'))

    # train VARMAX model with previously derived (p,q) values
    # use the model which gives smallest MSE on the validation set
    returns_mses, models = [], []
    print('Grid Search Started')
    start = time.time()
    for i in pq:
        if i[0]==0 and i[2]==0:
            pass
        else:
            print(f' Running for {i}')
            model = VARMAX(train_df, order=(i[0],i[2])).fit(disp=False)
            models.append(model)
            result = model.forecast(steps = val_df.shape[0]).reset_index(drop=True)
            returns_mse = ((result['returns'] - val_df['returns'])**2).mean()
            returns_mses.append(returns_mse)
            print(f' MSE for returns is {returns_mse}')
    end = time.time()
    print(f' Total time taken to complete grid search in seconds: {(end - start)}')
    print(f' Smallest MSE achieved at (p, d, q) values: {pq[np.argmin(returns_mses)]}')
    model = models[np.argmin(returns_mses)]
    pickle.dump(model, open('model.pkl', 'wb'))
    return 'model.pkl'


def modelForecast(testFilename, modelPath):
    """
    Predict returns using a fitted model.

    Args:
        testFilename (str): path to test data. The data will be in
            the same format as the supplied `data.csv` file
        parameters (any): fitted model and any additional parameters
            you need to make your forecasts

    Returns:
        forecast (np.array): vector of predictions. Make sure to
            return a prediction for each data point in the test set

    """
    test = pd.read_csv(testFilename)
    model = pickle.load(open(modelPath, 'rb'))
    # calculate the returns of X using the underlying method of calculating 'returns'
    get_dates(test, 'timestamp')
    test['xreturns'] = cal_returns(test, 'date', 'xprice')
    test_df = test[['returns', 'xreturns']]
    pred = model.forecast(steps = test_df.shape[0]).reset_index(drop=True)
    return pred['returns']


In [8]:
modelEstimate('train.csv')

The method to calculate xreturns seems to be the same as the underlying method to calculate returns.
before testing
The calculated xreturns passes the ADF test.
The given returns passes the ADF test.
after testing
df defined
The df ["returns", "xreturns"] passes the Johansen cointegration test.
In auto_arima, searching order of p and q for : returns
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-1032919.491, Time=17.87 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-156384.235, Time=7.11 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1032887.334, Time=13.07 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-451543.512, Time=22.62 sec


/opt/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-1032972.478, Time=9.49 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=-1032920.032, Time=18.25 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=-1032932.209, Time=19.45 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-1032930.969, Time=26.04 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=-1032933.242, Time=23.82 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=-1032933.275, Time=20.57 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=inf, Time=166.35 sec


/opt/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-1032970.604, Time=24.61 sec

Best model:  ARIMA(2,0,1)(0,0,0)[0]          
Total fit time: 369.280 seconds
optimal order for:returns is: (2, 0, 1) 


In auto_arima, searching order of p and q for : xreturns
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-1140867.579, Time=15.84 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-282420.193, Time=7.42 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-1140685.470, Time=13.56 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-572922.649, Time=24.35 sec


/opt/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-1140868.663, Time=21.53 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=-1140868.871, Time=16.04 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=-1140869.166, Time=21.96 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=-1140869.989, Time=26.31 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=-1140876.927, Time=30.07 sec
 ARIMA(6,0,0)(0,0,0)[0]             : AIC=-1140899.033, Time=41.53 sec
 ARIMA(7,0,0)(0,0,0)[0]             : AIC=-1140961.225, Time=46.35 sec
 ARIMA(7,0,1)(0,0,0)[0]             : AIC=-1140937.263, Time=40.93 sec
 ARIMA(6,0,1)(0,0,0)[0]             : AIC=-1140876.004, Time=51.22 sec
 ARIMA(7,0,0)(0,0,0)[0] intercept   : AIC=-1140961.228, Time=107.85 sec
 ARIMA(6,0,0)(0,0,0)[0] intercept   : AIC=-1140898.972, Time=92.73 sec
 ARIMA(7,0,1)(0,0,0)[0] intercept   : AIC=-1140937.235, Time=50.31 sec
 ARIMA(6,0,1)(0,0,0)[0] intercept   : AIC=-1140875.922, Time=41.47 sec

Best model:  ARIMA(7,0,0)(0,0,0)[0] intercept
Total fit time: 649.502 secon

/opt/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/tsa/statespace/varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)


 MSE for returns is nan
 Running for (7, 0, 0)
 MSE for returns is nan
 Total time taken to complete grid search in seconds: 497.84456992149353
 Smallest MSE achieved at (p, d, q) values: (2, 0, 1)


'model.pkl'

In [26]:
test = pd.read_csv('test.csv')
pred = modelForecast('test.csv', 'model.pkl')
true = test['returns']
print('The out-of-sample mean squared prediction error is ', ((pred-true)**2).mean())

The out-of-sample mean squared prediction error is  0.01829155203332383
